In [1]:
import os
import numpy as np
from scipy.misc import imread, imresize
import tensorflow as tf
import matplotlib.pyplot as plt
import random

tf.set_random_seed(777)

In [2]:
img_width = 64
img_height = 64

tfrecord_train = 'train.tfrecord'
tfrecord_test = 'test.tfrecord'
tfrecord_dir = 'tfrecords'

In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 10
n_train = 160
n_test = 40
n_class = 2

In [4]:
def read_and_decode(filename_queue, n_batch):
    
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    
    features = tf.parse_single_example(
        serialized_example,
        features={            
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        })
    
    # Convert from a scalar string tensor
    image = tf.decode_raw(features['image'], tf.uint8)        
    label = tf.cast(features['label'], tf.int32)
    label_onehot = tf.one_hot(label, depth=n_class)
    
    image = tf.reshape(image, [img_height, img_width, 3])    
    
    images, labels = tf.train.batch([image, label_onehot],
                                           batch_size=n_batch,
                                           capacity=10000,
                                           num_threads=4)
    
    return images, labels

In [5]:
keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, img_height, img_width, 3])
Y = tf.placeholder(tf.float32, [None, 2])

In [6]:
#L1 ImgIn shape=(?, 64, 64, 3)
W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# L2 ImgIn shape=(?, 32, 32, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# L3 ImgIn shape=(?, 16, 16, 64)
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[
                    1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 128 * 8 * 8])

In [7]:
# L4 FC 8x8x128 inputs -> 512 outputs
W4 = tf.get_variable("W4", shape=[128 * 8 * 8, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

# L5 Final FC 512 inputs -> 2 outputs
W5 = tf.get_variable("W5", shape=[512, n_class],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([n_class]))
logits = tf.matmul(L4, W5) + b5

In [8]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [9]:
cwd = os.getcwd()
train_path = os.path.join(cwd, tfrecord_dir, tfrecord_train)
test_path = os.path.join(cwd, tfrecord_dir, tfrecord_test)

In [10]:
filename_queue = tf.train.string_input_producer([train_path], num_epochs=training_epochs)
image_batch, label_batch = read_and_decode(filename_queue, batch_size)
filename_queue_test = tf.train.string_input_producer([test_path], num_epochs=1)
image_test, label_test = read_and_decode(filename_queue_test, n_test)

In [11]:
init_op = tf.group(tf.global_variables_initializer(),
                      tf.local_variables_initializer())
sess = tf.Session()
sess.run(init_op)

In [12]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(n_train / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = sess.run([image_batch, label_batch])
        batch_xs = batch_xs/255.
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 0001 cost = 0.946553156
Epoch: 0002 cost = 0.793260776
Epoch: 0003 cost = 0.774894100
Epoch: 0004 cost = 0.797910705
Epoch: 0005 cost = 0.758484304
Epoch: 0006 cost = 0.763750713
Epoch: 0007 cost = 0.799558040
Epoch: 0008 cost = 0.770335522
Epoch: 0009 cost = 0.762408720
Epoch: 0010 cost = 0.730643507
Epoch: 0011 cost = 0.865094427
Epoch: 0012 cost = 0.765212599
Epoch: 0013 cost = 0.798024461
Epoch: 0014 cost = 0.737989601
Epoch: 0015 cost = 0.764530282
Epoch: 0016 cost = 0.709436145
Epoch: 0017 cost = 0.772390585
Epoch: 0018 cost = 0.766136998
Epoch: 0019 cost = 0.736108653
Epoch: 0020 cost = 0.729286881
Epoch: 0021 cost = 0.703121480
Epoch: 0022 cost = 0.729869016
Epoch: 0023 cost = 0.701436047
Epoch: 0024 cost = 0.669311460
Epoch: 0025 cost = 0.615139045
Epoch: 0026 cost = 0.629877109
Epoch: 0027 cost = 0.647754641
Epoch: 0028 cost = 0.646531584
Epoch: 0029 cost = 0.538095472
Epoch: 0030 cost = 0.594856504
Epoch: 0031 cost = 0.532842956
Ep

In [13]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
test_xs, test_ys = sess.run([image_test, label_test])
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: test_xs, Y: test_ys, keep_prob: 1}))

# Get one and predict
r = random.randint(0, n_test - 1)
print("Label: ", sess.run(tf.argmax(test_ys[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: test_xs[r:r + 1], keep_prob: 1}))

Accuracy: 0.55
Label:  [0]
Prediction:  [1]


In [14]:
coord.request_stop()
coord.join(threads) 